In [1]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd


In [2]:
url='http://mops.twse.com.tw/mops/web/t51sb01'

In [3]:
payload = {'encodeURIComponent':'1',
          'step':'1',
          'firstin':'1',
          'TYPEK':'sii',
          'code':"02"}

In [4]:
# get the session
s = requests.Session()
r = s.get(url)

In [5]:
def get_table(s, url, payload):

    # use payload 
    response = s.post(url,data=payload)
    res = response.content
    soup = BeautifulSoup(res, 'html.parser')
    table = soup.find(id='table01')
    return table

In [6]:
def get_df(table):
    raws = table.find_all('table')[1].find_all('tr')
    # get the header 
    header = raws[0].find_all('th')
    header_of_table = [x.get_text() for x in header]
    # get the cell
    list_of_talbe=[]
    for raw in raws:
        r =  [x.get_text() for x in raw.find_all('td')]
        if len(r) > 0:
            list_of_talbe.append(r)
            
    df = pd.DataFrame(list_of_talbe,columns=header_of_table)
    return df

In [7]:
if r.ok:
    table = get_table(s, url, payload)
    df = get_df(table)

In [8]:
df

,公司代號,公司名稱,住址,營利事業統一編號,董事長,總經理,發言人,發言人職稱,代理發言人,總機電話,...,過戶地址,簽證會計師事務所,簽證會計師1,簽證會計師2,英文簡稱,英文通訊地址,傳真機號碼,電子郵件信箱,公司網址,公司網站內利害關係人專區網址
0,1201,味全食品工業股份有限公司,台北市松江路125號,11347802,陳永清,蘇守斌,張美凰,資深協理,李怡青,(02)25065020,...,台北市大同區承德路3段210號地下室一樓,資誠聯合會計師事務所,張祚誠,張明輝,Wei Chuan,"10F., No.125, Sung Chiang Rd.Taipei, Taiwan(R....",(02)25063182,service@weichuan.com.tw,http://www.weichuan.com.tw,http://www.weichuan.com.tw/investor
1,1203,味王股份有限公司,台北市中山北路二段79號5樓,07067309,陳清福,陳恭平,魏璟雄,經理,李兆弘,(02)25717271,...,台北市敦化南路二段97號地下2樓,大中國際聯合會計師事務所,林寬照,徐靖賢,VE WONG,"5TH FL., 79 CHUNG SHAN N. RD., SEC.2TAIPEI TAIWAN",(02)25811006,wei@vewong.com.tw,http://www.vewong.com,http://www.vewong.com/ec99/ushop20069/profile_...
2,1210,大成長城企業股份有限公司,台南市永康區蔦松二街三號,73008303,韓家宇,莊坤炎,周叔恆,特助,常岡德,06-2531111,...,台北市重慶南路一段八十三號六樓,安侯建業會計師事務所,李逢暉,陳眉芳,GREATWALL,3 NIAOSUN 2TH STREETYONGKONG TAINAN CONTY TAIWAN,06-2534596,john@greatwall-group.com,http://www.dachan.com,https://www.dachan.com/forward4?subChannelId=7...
3,1213,大西洋飲料股份有限公司,新北市三重區重陽路三段99號10樓,35012903,江國貴,孫幼英,楊克鳳,經理,洪松輝,(02)2982-0061,...,新北市三重區重陽路三段99號10樓,資誠聯合會計師事務所,阮呂曼玉,徐聖忠,N,"10F,99,Sec.3,Chung Yang Road,San Chung City,Ta...",(02)2984-1154,c5012903@ms23.hinet.net,http://www.applesidra.com.tw,http://www.applesidra.com.tw/apple7.html
4,1215,台灣卜蜂企業股份有限公司,台北市松江路八十七號十七樓,36514783,鄭武樾,許瑞通,陳銘龍,特別助理,林明芳,02-25077071,...,台北市重慶南路一段八十三號五樓,資誠聯合會計師事務所,王輝賢,謝智政,CPE,"17F., 87 Sung Chiang Road,Taipei, Taiwan 104",02-25064137,service@cptwn.com.tw,http://www.cptwn.com.tw,http://www.cptwn.com.tw/cpwebsite/profile6.html
5,1216,統一企業股份有限公司,71001台南市永康區中正路301號,73251209,羅智先,侯榮隆,蘇崇銘,副總經理,涂忠正,(06)253-2121,...,10570台北市松山區東興路8號統一大樓地下室一樓,資誠聯合會計師事務所,林姿妤,周建宏,UNI-PRESIDENT,"No.301, Zhongzheng Rd., Yongkang Dist.,Tainan ...",(06)253-2661,jlian@mail.pec.com.tw,http://www.uni-president.com.tw,http://www.uni-president.com.tw/
6,1217,愛之味股份有限公司,嘉義縣民雄鄉工業二路11號,65534366,陳哲芳,陳志展,陳哲芳,董事長,陳鏡仁 陳志展 蔡慧倫,05-2211521-4,...,台北市大安區敦化南路二段97號地下二樓,國富浩華聯合會計師事務所,黃鈴雯,劉奕宏,AGV,"NO.11,Kung Yeh Second Rd.,Min Hsiong Industria...",05-2216287,chingyi@mail.agv.com.tw,www.agv.com.tw,www.agv.com.tw
7,1218,泰山企業股份有限公司,彰化縣田中鎮興工路6號,59661701,詹逸宏,詹景超,林俐婉,副總經理,盧界煥,04-8742211,...,台北市敦化南路二段97號B2,安侯建業聯合會計師事務所,簡蒂暖,池世欽,TAISUN,"10,F1,NO.99,SEC.2,CHANG-ANE.RD.Taipei,Taiwan,R...",04-8758989,taisun1@taisun.com.tw,www.taisun.com.tw,http://www.taisun.com.tw/stakeholder.aspx
8,1219,福壽實業股份有限公司,台中市沙鹿區沙田路45號,56192402,洪堯昆,陳育信,張繡錦,協理,王四全,04-26362111,...,100台北市重慶南路一段2號5樓,群智聯合會計師事務所,黃奕睿,陳郁惠,FWUSOW,"45,Sha-Tyan,Sha-LuTaichung Taiwan R.O.C",04-26358452,fwusow@fwusow.com.tw,www.fwusow.com.tw,http://www.fwusow.com.tw/information/itemlist/...
9,1220,台榮產業股份有限公司,台北市中山區南京東路二段206號6樓,03543210,陳維讓,陳英凱,陳玉能,副總經理,黃棣楨,(02)2506-9521,...,台北市重慶南路一段83號5樓,立本台灣聯合會計師事務所,柯俊輝,張鈞鈞,TAIROUN,"6TH FL. NO. 206. NANKING E. RD., SEC. 2Taipei...",(02)2507-7020,glen6572@tairoun.com.tw,http://www.tairoun.com.tw/,建置中
